# Práticas e Técnicas Laboratoriais - MEB 2023/2024
## David Castanho Terroso - up202308694
### Tennakoon et al. 2018

#### Preprocessing RETOUCH images

(only the ones from Train since the ones from Test cannot integrate the test-train split, since there is no segmentation available)

In [1]:
from prepare_data import preprocess_oct_images
preprocess_oct_images()

#### Creating test-train split from the processed images

In [2]:
from prepare_data import create_test_train_set
create_test_train_set()

generating new test train SPLIT
{('Topcon',): 18, ('Cirrus',): 16, ('Spectralis',): 18}
{('Spectralis',): 6, ('Topcon',): 4, ('Cirrus',): 8}
{('Topcon',): 18, ('Spectralis',): 18, ('Cirrus',): 16}
{('Spectralis',): 6, ('Cirrus',): 8, ('Topcon',): 4}


A new test-train split from the processed images is used

In [1]:
from nutsflow import Shuffle, Collect, WriteCSV
from nutsml import SplitRandom, ReadPandas
import pandas as pd

DATA_ROOT = "..\RETOUCHdata\pre_processed\\"

print ('generating new test train SPLIT')
# reading training data
train_file = DATA_ROOT + 'slice_gt.csv'
data = ReadPandas(train_file, dropnan=True)
data = data >> Shuffle(7000) >> Collect()

# Split the data set into train and test sets with all the slices from the same volume remaining in one split
same_image = lambda s: s[0]
train_data, val_data = data >> SplitRandom(ratio=0.75, constraint=same_image)

col_names = ['image_name', 'vendor', 'root', 'slice', 'is_IRF', 'is_SRF', 'is_PED']
df_train = pd.DataFrame(train_data, columns=col_names)
df_val = pd.DataFrame(val_data, columns=col_names)

df_train.to_csv('./outputs/train_data_.csv', index=False)
df_val.to_csv('./outputs/test_data_.csv', index=False)

# based on retouch-classify, adapted to fit the headers

generating new test train SPLIT


"\nwriter = WriteCSV('./outputs/train_data_.csv', skipheader=0)\ntrain_data >> writer\n\nwriter = WriteCSV('./outputs/test_data_.csv', skipheader=0)\nval_data >> writer\n"

#### Creating ROI masks

In [2]:
from prepare_data import create_roi_masks
create_roi_masks()

C:\Users\david\AppData\Local\Temp\ipykernel_15996\2547971702.py:2: UserWarning: Possible precision loss converting image of type float32 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  create_roi_masks()


Cirrus_TRAIN001_000.tiff 37
Cirrus_TRAIN001_001.tiff 36
Cirrus_TRAIN001_002.tiff 34
Cirrus_TRAIN001_003.tiff 36
Cirrus_TRAIN001_004.tiff 38
Cirrus_TRAIN001_005.tiff 38
Cirrus_TRAIN001_006.tiff 38
Cirrus_TRAIN001_007.tiff 38


KeyboardInterrupt: 

#### Training U-Net segmentation model

In [1]:
# Needs the last cell to run in order to implement it
# Needs build_batch_train from train_model and validate the decisions made with the supervisors
# Classify or segment only? Has implications on the data selected to train since segmentation can be extended to other datasets 
# and classification can not. Done by changing the hyper parameters

from train_model import train_model
train_model()

Using existing test train SPLIT


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0 (Conv2D)    │ (None, 256, 128,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1 (Conv2D)    │ (None, 254, 126,  │     36,928 │ conv1_0[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2 (Conv2D)    │ (None, 252, 124,  │     36,928 │ conv1_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1                 │ (None, 252, 124,  │        256 │ conv1_2[0][0]     │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 252, 124,  │          0 │ bn1[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 126, 62,   │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv2D)    │ (None, 124, 60,   │     73,856 │ pool1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv2D)    │ (None, 122, 58,   │    147,584 │ conv2_1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2                 │ (None, 122, 58,   │        512 │ conv2_2[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 122, 58,   │          0 │ bn2[0][0]         │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool2               │ (None, 61, 29,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_1 (Conv2D)    │ (None, 59, 27,    │    295,168 │ pool2[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_2 (Conv2D)    │ (None, 57, 25,    │    590,080 │ conv3_1[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn3                 │ (None, 57, 25,    │      1,024 │ conv3_2[0][0]     │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 57, 25,    │          0 │ bn3[0][0]         │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool3               │ (None, 28, 12,    │          0 │ activation_2[0][

 Total params: 7,781,828 (29.69 MB)

 Trainable params: 7,779,140 (29.68 MB)

 Non-trainable params: 2,688 (10.50 KB)

Starting network training
Training Epoch 0
(16, 164, 36, 4)
(16, 164, 36, 4)


KeyboardInterrupt: 

#### Training the adversarial model

In [1]:
from train_adversarial import train_model
train_model()

# NOTE: LOAD_WEIGHTS set as true in this case, in order to load the weights from the previous cell
# Needs build_batch_train from train_model and validate the decisions made with the supervisors

Using existing test train SPLIT


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 164, 36,   │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 164, 36,   │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0_ (Conv2D)   │ (None, 164, 36,   │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0_m (Conv2D)  │ (None, 164, 36,   │     12,608 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1_ (Conv2D)   │ (None, 162, 34,   │     36,928 │ conv1_0_[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1_m (Conv2D)  │ (None, 162, 34,   │     36,928 │ conv1_0_m[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2_ (Conv2D)   │ (None, 160, 32,   │     36,928 │ conv1_1_[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2_m (Conv2D)  │ (None, 160, 32,   │     36,928 │ conv1_1_m[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1_                │ (None, 160, 32,   │        256 │ conv1_2_[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1_m               │ (None, 160, 32,   │        256 │ conv1_2_m[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 160, 32,   │          0 │ bn1_[0][0]        │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 160, 32,   │          0 │ bn1_m[0][0]       │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ merge_in            │ (None, 160, 32,   │          0 │ activation[0][0], │
│ (Concatenate)       │ 128)              │            │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 80, 16,    │          0 │ merge_in[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv2D)    │ (None, 78, 14,    │    147,584 │ pool1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv2D)    │ (None, 76, 12,    │    147,584 │ conv2_1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2                 │ (None, 76, 12,    │        512 │ conv2_2[0][0]   

 Total params: 4,895,298 (18.67 MB)

 Trainable params: 4,893,250 (18.67 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0 (Conv2D)    │ (None, 256, 128,  │      9,472 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1 (Conv2D)    │ (None, 254, 126,  │     36,928 │ conv1_0[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2 (Conv2D)    │ (None, 252, 124,  │     36,928 │ conv1_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1                 │ (None, 252, 124,  │        256 │ conv1_2[0][0]     │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 252, 124,  │          0 │ bn1[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 126, 62,   │          0 │ activation_5[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv2D)    │ (None, 124, 60,   │     73,856 │ pool1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv2D)    │ (None, 122, 58,   │    147,584 │ conv2_1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2                 │ (None, 122, 58,   │        512 │ conv2_2[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 122, 58,   │          0 │ bn2[0][0]         │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool2               │ (None, 61, 29,    │          0 │ activation_6[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_1 (Conv2D)    │ (None, 59, 27,    │    295,168 │ pool2[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_2 (Conv2D)    │ (None, 57, 25,    │    590,080 │ conv3_1[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn3                 │ (None, 57, 25,    │      1,024 │ conv3_2[0][0]     │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 57, 25,    │          0 │ bn3[0][0]         │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool3               │ (None, 28, 12,    │          0 │ activation_7[0][

 Total params: 7,781,828 (29.69 MB)

 Trainable params: 7,779,140 (29.68 MB)

 Non-trainable params: 2,688 (10.50 KB)

AssertionError: 

#### Testing model

In [2]:
from test_model import test_model
test_model()

# Useless and problematic code was commented
# Needs build_batch_train from train_model and validate the decisions made with the supervisors

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 256, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0 (Conv2D)    │ (None, 256, 512,  │      9,472 │ input_layer_4[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1 (Conv2D)    │ (None, 254, 510,  │     36,928 │ conv1_0[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2 (Conv2D)    │ (None, 252, 508,  │     36,928 │ conv1_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1                 │ (None, 252, 508,  │        256 │ conv1_2[0][0]     │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 252, 508,  │          0 │ bn1[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 126, 254,  │          0 │ activation_11[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv2D)    │ (None, 124, 252,  │     73,856 │ pool1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv2D)    │ (None, 122, 250,  │    147,584 │ conv2_1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2                 │ (None, 122, 250,  │        512 │ conv2_2[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_12       │ (None, 122, 250,  │          0 │ bn2[0][0]         │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool2               │ (None, 61, 125,   │          0 │ activation_12[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_1 (Conv2D)    │ (None, 59, 123,   │    295,168 │ pool2[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_2 (Conv2D)    │ (None, 57, 121,   │    590,080 │ conv3_1[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn3                 │ (None, 57, 121,   │      1,024 │ conv3_2[0][0]     │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_13       │ (None, 57, 121,   │          0 │ bn3[0][0]         │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool3               │ (None, 28, 60,    │          0 │ activation_13[0]

 Total params: 7,781,828 (29.69 MB)

 Trainable params: 7,779,140 (29.68 MB)

 Non-trainable params: 2,688 (10.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 256, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_0 (Conv2D)    │ (None, 256, 512,  │      9,472 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_1 (Conv2D)    │ (None, 254, 510,  │     36,928 │ conv1_0[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_2 (Conv2D)    │ (None, 252, 508,  │     36,928 │ conv1_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn1                 │ (None, 252, 508,  │        256 │ conv1_2[0][0]     │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_17       │ (None, 252, 508,  │          0 │ bn1[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 126, 254,  │          0 │ activation_17[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_1 (Conv2D)    │ (None, 124, 252,  │     73,856 │ pool1[0][0]       │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_2 (Conv2D)    │ (None, 122, 250,  │    147,584 │ conv2_1[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2                 │ (None, 122, 250,  │        512 │ conv2_2[0][0]     │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 122, 250,  │          0 │ bn2[0][0]         │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool2               │ (None, 61, 125,   │          0 │ activation_18[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_1 (Conv2D)    │ (None, 59, 123,   │    295,168 │ pool2[0][0]       │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3_2 (Conv2D)    │ (None, 57, 121,   │    590,080 │ conv3_1[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn3                 │ (None, 57, 121,   │      1,024 │ conv3_2[0][0]     │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_19       │ (None, 57, 121,   │          0 │ bn3[0][0]         │
│ (Activation)        │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool3               │ (None, 28, 60,    │          0 │ activation_19[0]

 Total params: 7,781,828 (29.69 MB)

 Trainable params: 7,779,140 (29.68 MB)

 Non-trainable params: 2,688 (10.50 KB)

AssertionError: 